In [19]:
# Justin Harper
# WSU ID: 10696738

## Programming Assignment 2 - Part 1
### Cpts 355 - Spring 2016
### An Interpreter for a Postscript-like Language

### Assigned Feb. 3, 2016

### Due Friday, Feb. 12, 2016
Develop your code in a file named `sps.ipynb`, starting from this notebook file. When you are finished, upload `sps.ipynb` on the course Turnin Page. 

The entire interpreter project (Parts 1 and Part 2 together) will count for 10% of your course grade. This first part is worth 20% of that 10%: the intention is to make sure that you are on the right track and have a chance for mid-course correction before completing Part 2. However, note that the work and amount of code involved in Part 1 is a large fraction of the total project, so you need to get going on this part right away.

### This assignment is to be your own work. Refer to the course academic integrity statement in the syllabus.

## The problem
In this assignment you will write an interpreter in Python for a small PostScript-like language, concentrating on key computational features of the abstract machine, omitting all PS features related to graphics, and using a somewhat-simplified syntax.

The simplified language, SPS, has the following features of PS
* integer constants, e.g. `123`: in python3 there is no practical limit on the size of integers
* boolean constants, `true` and `false` (Note that the boolean constants in python are `True` and `False`)
* name constants, e.g. `/fact`: start with a `/` and letter followed by an arbitrary sequence of letters and numbers
* names to be looked up in the dictionary stack, e.g. `fact`: as for name constants, without the `/`
* code constants: code between matched curly braces `{` ... `}`
* built-in operators on numbers: `add`, `sub`, `mul`, `div`, `eq`, `lt`, `gt`
* built-in operators on boolean values: `and`, `or`, `not`; these take boolean operands only. Anything else is an error.
* built-in sequencing operators: `if`, `ifelse`; make sure that you understand the order of the operands on the stack. Play with ghostscript if necessary to help understand what is happening.
* stack operators: `dup`, `exch`, `pop`
* dictionary creation operator: `dict`; takes one operand from the operand stack, ignores it, and creates a new, empty dictionary on the operand stack
* dictionary stack manipulation operators: `begin`, `end`. `begin` requires one dictionary operand on the operand stack; `end` has no operands.
* name definition operator: `def`. This requires two operands, a name and a value
* defining (using `def`) and calling functions
* stack printing operator (prints contents of stack without changing it): `stack`
* top-of-stack printing operator (pops the top element of the stack and prints it): `=`


## Requirements for Part 1 (Due Feb. 12)
In Part 1 you will build some essential pieces of the interpreter but not yet the full interpreter. The pieces you build will be driven by Python test code rather than actual Postscript programs. The pieces you are going to build first are:
* The operand stack
* The dictionary stack
* The operators that don't involve code arrays: all of the operators except `if`, `ifelse`.
In Part 2 we will add the implementations for `if`, `ifelse`, calling functions, as well as interpreting input strings in the Postscript language.
* Looking up names

### The operand stack
The operand stack should be implemented as a Python list. The list will contain **Python** integers, booleans, and strings, and later in Part 2 code arrays. Python integers and booleans on the stack represent Postscript integers and booleans. Python strings on the stack represent names of Postscript variables (see the handling of names and the `def` operator below.

When using a list as a stack one of the decisions you have to make is where the *hot* end of the stack is located. (The *hot* end is where pushing and popping happens). Will the hot end be at position `0`, the head of the list, or at position `-1`, the end of the list? It's your choice.

### The dictionary stack
The dictionary stack is also implemented as a Python list. It will contain **Python** dictionaries which will be the implementation for **Postscript** dictionaries. The dictionary stack needs to support adding and removing dictionaries at the hot end, as well as defining and looking up names. 

### Operators
Operators will be implemented as zero-argument Python functions that manipulate the operand and dictionary stacks. For example, the `add` operator could be implemented as the Python function (with comments instead of actual implementations)
```
def add():
    op1 = # pop the top value off the operand stack
    op2 = # pop the top value off the operand stack
    # push (op1 + op2) onto the operand stack
```
You may run into conflicts for some of the names of these functions . For example, the function for the `not` operator can't be named `not` because it is reserved for another use in Python. So you could do something like:
```
def psnot():
    // pop the top value off the operand stack and push its negation onto the operand stack
```    
The `begin` and `end` operators are a little different in that they manipulate the dictionary stack in addition to or instead of the operand stack. Remember that the `dict` operator affects *only* the operand stack.

The `def` operator takes two operands from the operand stack: a string (recall that strings in the operand stack represent names of postscript variables) and a value. It changes the dictionary at the hot end of the dictionary stack so that the string is mapped to the value by that dictionary. Notice that `def` does ***not*** change the number of dictionaries on the dictionary stack!

### Name lookup

Name lookup is implemented by a Python function:
```
def lookup(name):
    # search the dictionaries on the dictionary stack starting at the hot end to find one that contains name
    # return the value associated with name
```
Note that name lookup is ***not*** a Postscript operator, but it ***is*** implemented by a Python function.

## Your Code Start Here

In [20]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []
# other globals for me to use
var = ''
brace = ''
curly = False


# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end. Recall that `pass` in python is a no-op: replace it with your code.

def opPop(): 
    return opstack.pop()

def opPush(value):
    opstack.append(value)

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [55]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dstack = [{}]

# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

def dictPop():
    a = dstack.pop()
    if(a == {}):
        dictPush(a)
        return {}
    else:
        return a
    

def dictPush(value):
    dstack.append(value)

def define(name, value):
    a = opstack.pop()
    b = opstack.pop()
    x = dstack.pop()
    x[b] = a
    dstack.append(x)

def lookup(name):
    # return the value associated with name
    # what is your design decision about what to do when there is no definition for name
    
    # need to determine if name is a number or a name
    #if name.isdigit():
    #    return name
    
    print("Lookup: name: " + name)
    print(dstack)
    
    if type(name) == type(1):    
        return name    
    else:
        # need to lookup the name in the dict        
        
        for a in dstack:
            if(a == {}):
                return None            
            return a[name]
        

In [22]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt

def Equal():
    lookup(opstack.pop())


def EQ():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == b):
        opstack.append("true")
    else:
        opstack.append("false")

def Add():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    sum = int(a) + int(b)
    opstack.append(sum)
    
def Mul():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    prod = int(a) * int(b)
    opstack.append(prod)

def Div():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 0):
        exit()
    else:
        quote = int(b) / int(a)
        opstack.append(quote)
    

def Lt():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(b < a):
        opstack.append("true")
    else:
        opstack.append("false")

def Gt():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(b > a):
        opstack.append("true")
    else:
        opstack.append("false")


In [23]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not
def And():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 'true' and b == 'true'):
        opstack.append('true')
    else:
        opstack.append('false')
        
def Not():
    a = lookup(opstack.pop())
    if(a == 'false'):
        opstack.append('true')
    else:
        opstack.append('false')
        
def Or():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 'true' or b == 'true'):
        opstack.append('true')
    else:
        opstack.append('false')

In [24]:
# Define the stack manipulation operators in this cell: dup, exch, pop
def Dup():
    a = opstack.pop()
    opstack.append(a)
    opstack.append(a)
    
def Exch():
    a = opstack.pop()
    b = opstack.pop()
    
    opstack.append(a)
    opstack.append(b)
def Pop():
    opstack.pop()

In [63]:
# Define the dictionary manipulation operators in this cell: dict, begin, end, def
# name the function for the def operator psDef because def is reserved in Python
def Dict():
    # takes one operand from the operand stack, ignores it, and creates a new, empty dictionary on the operand stack
    opstack.pop()
    dstack.append({})
    
def Begin(d):
    # begin pushes a dictionary on the dictionary stack
    dstack.append(d)
def End():
    #  end removes a dictionary from the dictionary stack
    dstack.pop()
    
def Def():
    # def creates or modifies an entry in the top dictionary on the dictionary stack.
    a = opstack.pop()
    b = opstack.pop()
    d = dstack.pop()
    d[str(b)] = a
    dstack.append(d)
    

In [26]:
# Define the IF ELSE and ELSEIF operators
# ELSE will be handled in processLine

def If():
    l = opstack.pop()
    b = opstack.pop()
    if b == 'true':
        processLine(l[1:-1])
        
        
def ElseIf():
    lf = opstack.pop()
    lt = opstack.pop()
    b = opstack.pop()
    
    if b == 'true':
        processLine(lt[1:-1])
    else:
        processLine(lf[1:-1])
        
    
    


In [45]:
# Define processLine
def processLine(line):
    global brace
    global var
    global curly
    
    for c in line.split():
        if c[0] >='0' and c[0] <= '9':
            opstack.append(int(c))
        elif c == '{':
            brace = c
            curly = True
        elif curly:
            brace +=' '
            brace += c
            if  c == '}':
                curly = False
                opstack.append(brace)
        elif c[0] == '/':
            var = c[1:]
            opstack.append(var)
        
        else:
            handleIt(c)

In [28]:
# Define the printing operators in this cell: =, stack
# Pick a good name for the code implementing =
def PrintOP():
    print(opstack)

def PrintD():
    print(dstack)
    
def PrintVal():
    a = lookup(opstack.pop())
    print (a)

In [61]:
# Define handleIt()

def handleIt(token):
    print(token)
    if token == '=': Equal()
    #do logical stuff
    elif token == 'and': And()
    elif token == 'or' : Or()
    elif token == 'eq' : Eq()
    elif token == 'not': Not()
    elif token == 'gt' : Gt()
    elif token == 'lt' : Lt()
    # do aritmatic stuff
    elif token == 'add': Add()
    elif token == 'sub': Sub()
    elif token == 'mul': Mul()
    elif token == 'div': Div()
        
    #not logical or math must be something else
    elif token == 'stack': PrintOP()
    elif token == 'if': If()
    elif token == 'ifelse': ElseIf()
    elif token == 'begin': Begin()
    elif token == 'dup': Dup()
    elif token == 'exch': Exch()
    elif token == 'pop': Pop()
    elif token == 'def': Def()
    elif token == 'end': End()
        
    else:
        s = lookup(str(token))
        print("s = " + s)
        if(s[0] == '{'):
            processLine(s[1:-1])
        else:
            stack.append(s)

In [64]:
PostScript = "/x 1 def"
PS2 = "x 3 eq {x 1 add} if"

PS = [PostScript, PS2]

for x in PS:
    print(x)
    processLine(x)
    
    
printOP()
printD()




/x 1 def
def
x 3 eq {x 1 add} if
x
Lookup: name: x
[{'x': 1}]


TypeError: Can't convert 'int' object to str implicitly

## Test your code
With all of that stuff defined, you will be able to test your interpreter using Python code like this:

In [27]:
def testAdd():
    opPush(1)
    opPush(2)
    Add()
    if opPop() != 3: return False   
    return True

def testLookup():
    opPush("n1")
    opPush(3)
    Def()
    if lookup("n1") != 3: return False
    return True


# go on writing test code for ALL of your code here; think about edge cases, and 
# other points where you are likely to make a mistake.

# now an easy way to run all the test cases and make sure that they all return true
# is

testCases = [testAdd, testLookup] # add the names of your test functions to this list
def testAll1():
    for test in testCases:
        if not test(): return False
    return True

# but wouldn't it be nice to run all the tests, instead of stopping on the first failure,
# and see which ones failed
# How about something like:

testCases = [('add', testAdd), ('lookup', testLookup)] # add you test functions to this list along with suitable names
def testAll2():
    failedTests = [testName for (testName, testProc) in testCases if not testProc()]
    if failedTests:
        return ('Some tests failed', failedTests)
    else: return ('All tests OK')
    


In [55]:
testAll2()

'All tests OK'